In [ ]:
import calendar
import json
import math
import numpy
import os
import pandas
import pyarrow
import sys

from datetime import date, datetime
from os import path
from dotenv import load_dotenv
from sqlalchemy import create_engine

# Steps to install
# 1. pip install sqlalchemy-bigquery google-cloud-bigquery-storage pyarrow
# 2. Copy the credentials file to wherever you set BIGQUERY_CREDENTIALS_PATH to

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')
engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))

In [ ]:
sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
import create_queries
%load_ext autoreload
%autoreload 2

In [ ]:
COMPANY_IDENTIFIER = 'SO'
COMPANY_NAME = COMPANY_IDENTIFIER
TRANSFER_PACKAGES_START_DATE = '2020-01-01'
SALES_TRANSACTIONS_START_DATE = '2020-01-01'
ANALYSIS_PARAMS = {
    'sold_threshold': 1.0,
    'find_parent_child_relationships': False,
    'use_prices_to_fill_missing_incoming': True,
    'external_pricing_data_config': {
        'category_to_fixed_prices': {
           "Capsule (weight - each)":{
              "Each":23.594097452934655
           },
           "Clone - Cutting":{
              "Each":2.153121902874133
           },
           "Clone - Tissue Culture":{
              "Each":0.0
           },
           "Edible (volume - each)":{
              "Each":6.146205207527713
           },
           "Edible (weight - each)":{
              "Each":7.5740532898741435
           },
           "Extract (volume - each)":{
              "Each":11.43294776119403
           },
           "Extract (weight - each)":{
              "Each":9.832834338863785
           },
           "Extract (weight)":{
              "Grams":23.60053333333333,
              "Pounds":10705.013115733333
           },
           "Flower":{
              "Grams":0.6615655299919327,
              "Pounds":0.0
           },
           "Flower (packaged - each)":{
              "Each":6.62190019193858
           },
           "Flower (packaged eighth - each)":{
              "Each":7.620680869582421
           },
           "Flower (packaged gram - each)":{
              "Each":0.36474099339160093
           },
           "Flower (packaged half ounce - each)":{
              "Each":17.340007024938533
           },
           "Flower (packaged ounce - each)":{
              "Each":72.03456896551724
           },
           "Flower (packaged quarter - each)":{
              "Each":14.970933920704848
           },
           "Fresh Cannabis Plant":{
              "Pounds":0.0
           },
           "Immature Plant":{
              "Each":0.6451699946033459
           },
           "Leaf":{
              "Grams":0.0,
              "Pounds":0.0
           },
           "Other Concentrate (volume - each)":{
              "Each":10.0
           },
           "Other Concentrate (weight - each)":{
              "Each":9.10354624425141
           },
           "Other Concentrate (weight)":{
              "Grams":0.0
           },
           "Pre-Roll Flower":{
              "Each":1.2455980167454037
           },
           "Pre-Roll Infused":{
              "Each":3.4070715249662618
           },
           "Pre-Roll Leaf":{
              "Each":0.2721679544530297
           },
           "Seeds":{
              "Grams":0.0,
              "Ounces":0.0,
              "Pounds":0.0
           },
           "Seeds (each)":{
              "Each":13.81578947368421
           },
           "Shake":{
              "Pounds":0.0
           },
           "Shake (Packaged Half Ounce - each)":{
              "Each":13.5
           },
           "Shake (Packaged Quarter - each)":{
              "Each":0.01
           },
           "Tincture (volume - each)":{
              "Each":18.740093427835063
           },
           "Topical (volume - each)":{
              "Each":15.084248927038628
           },
           "Topical (weight - each)":{
              "Each":6.851502702702704
           },
           "Vape Cartridge (volume - each)":{
              "Each":19.749874823014572
           },
           "Vape Cartridge (weight - each)":{
              "Each":16.83838680865686
           }
        }
    }
}

In [ ]:
# Download packages, sales transactions, incoming / outgoing tranfers
company_incoming_transfer_packages_query = create_queries.create_company_incoming_transfer_packages_query(COMPANY_IDENTIFIER, TRANSFER_PACKAGES_START_DATE)
company_outgoing_transfer_packages_query = create_queries.create_company_outgoing_transfer_packages_query(COMPANY_IDENTIFIER, TRANSFER_PACKAGES_START_DATE)
company_sales_receipts_query = create_queries.create_company_sales_receipts_query(COMPANY_IDENTIFIER, SALES_TRANSACTIONS_START_DATE)
company_sales_transactions_query = create_queries.create_company_sales_transactions_query(COMPANY_IDENTIFIER, SALES_TRANSACTIONS_START_DATE)
company_inventory_packages_query = create_queries.create_company_inventory_packages_query(
    COMPANY_IDENTIFIER,
    include_quantity_zero=True,
)

company_incoming_transfer_packages_dataframe = pandas.read_sql_query(company_incoming_transfer_packages_query, engine)
company_outgoing_transfer_packages_dataframe = pandas.read_sql_query(company_outgoing_transfer_packages_query, engine)
company_sales_receipts_dataframe = pandas.read_sql_query(company_sales_receipts_query, engine)
company_sales_transactions_dataframe = pandas.read_sql_query(company_sales_transactions_query, engine)
company_inventory_packages_dataframe = pandas.read_sql_query(company_inventory_packages_query, engine)

In [ ]:
sys.path.append(path.realpath(path.join(os.getcwd(), "../../scripts/analysis")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))

from bespoke.inventory.analysis import active_inventory_util as util
from bespoke.inventory.analysis import inventory_valuations_util as valuations_util

In [ ]:
sql_helper = util.BigQuerySQLHelper(engine)

d = util.Download()
d.download_dataframes(
    incoming_transfer_packages_dataframe=company_incoming_transfer_packages_dataframe,
    outgoing_transfer_packages_dataframe=company_outgoing_transfer_packages_dataframe,
    sales_receipts_dataframe=company_sales_receipts_dataframe,
    sales_transactions_dataframe=company_sales_transactions_dataframe,
    inventory_packages_dataframe=company_inventory_packages_dataframe,
    sql_helper=sql_helper,
)

In [ ]:
TODAY_DATE = date.today().strftime('%Y-%m-%d')
print(f'Today is {TODAY_DATE}')

In [ ]:
company_incoming_transfer_packages_dataframe['created_month'] = pandas.to_datetime(company_incoming_transfer_packages_dataframe['created_date']).dt.strftime('%Y-%m')
unique_incoming_transfer_package_months = company_incoming_transfer_packages_dataframe['created_month'].unique()

In [ ]:
company_sales_receipts_dataframe['sales_month'] = pandas.to_datetime(company_sales_receipts_dataframe['sales_datetime']).dt.strftime('%Y-%m')
unique_company_sales_receipt_months = company_sales_receipts_dataframe['sales_month'].unique()

In [ ]:
aggregate_unique_months = []
for month in unique_incoming_transfer_package_months:
    if month not in aggregate_unique_months:
        aggregate_unique_months.append(month)
for month in unique_company_sales_receipt_months:
    if month not in aggregate_unique_months:
        aggregate_unique_months.append(month)
aggregate_unique_months.sort()

unique_inventory_dates = []
for month in aggregate_unique_months:
    date_object = datetime.strptime(month, '%Y-%m')
    date_object = date_object.replace(day = calendar.monthrange(date_object.year, date_object.month)[1])
    eom_date_str = datetime.strftime(date_object, '%Y-%m-%d')
    if eom_date_str < TODAY_DATE:
        unique_inventory_dates.append(eom_date_str)

unique_inventory_dates.append(TODAY_DATE)
unique_inventory_dates = [datetime.strftime(datetime.strptime(unique_inventory_date, '%Y-%m-%d'), '%m/%d/%Y') for unique_inventory_date in unique_inventory_dates]

INVENTORY_DATES = unique_inventory_dates

In [ ]:
q = util.Query()
q.inventory_dates = INVENTORY_DATES
q.company_name = COMPANY_NAME

id_to_history = util.get_histories(d, params=ANALYSIS_PARAMS)
util.print_counts(id_to_history)

In [ ]:
date_to_inventory_packages_dataframe = {}
inventory_valuations = []

for inventory_date in INVENTORY_DATES:
    computed_inventory_package_records = util.create_inventory_dataframe_by_date(
        id_to_history,
        inventory_date,
        params=ANALYSIS_PARAMS,
    )    
    computed_inventory_packages_dataframe = pandas.DataFrame(
        computed_inventory_package_records,
        columns=util.get_inventory_column_names(),
    )
    date_to_inventory_packages_dataframe[inventory_date] = computed_inventory_packages_dataframe
    inventory_valuations.append(valuations_util.get_total_valuation_for_date(
        computed_inventory_packages_dataframe=computed_inventory_packages_dataframe,
        company_incoming_transfer_packages_dataframe=company_incoming_transfer_packages_dataframe,
    ))

In [ ]:
from_packages_inventory_dataframe = company_inventory_packages_dataframe[[
    'package_id',
    'packaged_date',
    'unit_of_measure',
    'product_category_name',
    'product_name',
    'quantity',
]].sort_values('package_id')

package_id_to_actual_row = {}
for index, row in from_packages_inventory_dataframe.iterrows():
    package_id_to_actual_row[str(row['package_id'])] = row

res = util.compare_inventory_dataframes(
    computed=date_to_inventory_packages_dataframe[unique_inventory_dates[-1]],
    actual=from_packages_inventory_dataframe,
    options={
        'num_errors_to_show': 50,
        'accept_computed_when_sold_out': True
    }
)

In [ ]:
print(f'Plotting sales revenue vs cost-based inventory valuation for dates: {unique_inventory_dates}')
valuations_util.plot_inventory_and_revenue(
    q=q,
    sales_receipts_dataframe=d.sales_receipts_dataframe,
    inventory_valuations=inventory_valuations,
)